### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Changing Momentum

### For ImageNet as a source

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

pre_trained_model = InceptionV3(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [5]:
def compile_model(rate):
  from tensorflow.keras.optimizers import SGD

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(1000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(500, activation='relu')(x)                  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = SGD(lr=0.0001, momentum = rate), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [6]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [7]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [ ]:
rate = 0
for i in range(0, 10):
  print("******************* Training model for momentum of", rate, "************************")
  rate = rate + 0.1
  imagenet_inception_v3 = compile_model(rate)
  history = imagenet_inception_v3.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
  print("******************* Model evaluation for a momentum of", rate, "*************************")
  eval = imagenet_inception_v3.evaluate(X_test, y_test)
  print("******************* Train time is", sum(cb.logs), "***********")
  print("f1 score is", sum(eval[6])/15)

******************* Training model for momentum of 0 ************************


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 92s 414ms/step - loss: 2.7667 - acc: 0.0679 - auc_1: 0.5018 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7618 - val_acc: 0.0727 - val_auc_1: 0.5016 - val_recall_1: 3.3333e-04 - val_precision_1: 0.1250 - val_hamming_loss: 0.9997 - val_f1_score: 6.5359e-04
Epoch 2/10
200/200 [==============================] - 81s 403ms/step - loss: 2.7447 - acc: 0.0717 - auc_1: 0.5062 - recall_1: 4.1667e-05 - precision_1: 0.2000 - hamming_loss: 1.0000 - f1_score: 8.2355e-05 - val_loss: 2.7392 - val_acc: 0.0760 - val_auc_1: 0.5077 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
200/200 [==============================] - 79s 397ms/step - loss: 2.7348 - acc: 0.0708 - auc_1: 0.5095 - recall_1: 8.3333e-05 - precision_1: 0.3333 - hamming_loss: 0.9999 - f1_score: 1.6554e-04 - val_loss: 2.7272 - val_acc: 0.0673 - val_auc_1: